In [1]:
%pwd

'e:\\scaledge_projects\\bird_species_classification\\research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'e:\\scaledge_projects\\bird_species_classification'

In [43]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    testing_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_lr_rate: float

In [44]:
from src.bird_classification.utils import *
from src.bird_classification.utils.common import create_directories
# import tensorflow as tf
import torch
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import time

In [45]:
with open("src/bird_classification/utils/params.yaml", 'r') as file:
        params = yaml.safe_load(file)
        print(params)

with open("src/bird_classification/utils/config.yaml", 'r') as file:
        config = yaml.safe_load(file)
        print(config)

{'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 32, 'INCLUDE_TOP': False, 'EPOCHS': 2, 'CLASSES': 10, 'MOMENTUM': 0.9, 'LEARNING_RATE': 0.01}
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/Sibasis555/Face_mask_detection/raw/main/datasets/dummy_bird_species.zip', 'local_data_file': 'artifacts/data_ingestion/dummy_bird_species.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.pth', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.pth'}, 'prepare_callbacks': {'root_dir': 'artifacts/prepare_callbacks', 'tensorboard_root_log_dir': 'artifacts/prepare_callbacks/tensorboard_log_dir', 'checkpoint_model_filepath': 'artifacts/prepare_callbacks/checkpoint_dir/model.pth'}, 'training': {'train_data': 'artifacts/data_ingestion/dummy_bird_species/Train', 'te

In [46]:
class ConfigurationManager: #Config Entity data
    def __init__(self):
        get_config_and_param()
        self.config = config
        self.params = params
    
        create_directories([self.config["artifacts_root"]])
        # print(self.config)

    def get_training_config(self) -> TrainingConfig:
        training = self.config["training"]
        prepare_base_model = self.config["prepare_base_model"]
        params = self.params
        # print(self.config["training"])
        training_data = self.config["training"]["train_data"]
        testing_data = self.config["training"]["test_data"]
        create_directories([
            Path(training["root_dir"])
        ])
        
        training_config = TrainingConfig(
            root_dir=Path(training["root_dir"]),
            trained_model_path=Path(training["trained_model_path"]),
            updated_base_model_path=Path(prepare_base_model["updated_base_model_path"]),
            training_data=Path(training_data),
            testing_data=Path(testing_data),
            params_epochs=params["EPOCHS"],
            params_batch_size=params["BATCH_SIZE"],
            params_is_augmentation=params["AUGMENTATION"],
            params_image_size=params["IMAGE_SIZE"],
            params_lr_rate=params["LEARNING_RATE"]
        )

        return training_config

In [47]:
class ImageDataset(Dataset):
    def __init__(self, data_folder, transform):
        self.data_folder = data_folder
        self.transform = transform
        self.classes = os.listdir(data_folder)
        self.image_paths = []
        self.labels = []

        for class_idx, class_name in enumerate(self.classes):
            class_path = os.path.join(data_folder, class_name)
            images = os.listdir(class_path)
            self.image_paths.extend([os.path.join(class_path, img) for img in images])
            self.labels.extend([class_idx] * len(images))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self,index):
        image_path = self.image_paths[index]
        label = self.labels[index]

        image = Image.open(image_path).convert("RGB")

        if self.transform:
            # print('before',image.size)
            image = self.transform(image)
            # print('after',image.size)
        return image, label

In [48]:
class Data_preprocessing:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.train_path = self.config.training_data
        self.test_path = self.config.testing_data
        # print("path--",self.config.training_data)
        
    def data_normalization(self):
        data_transforms = transforms.Compose([
        transforms.Resize(size=(224,224), interpolation=Image.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
        ])
        #Getting the data
        # print(self.train_path)
        self.train_set = ImageDataset(self.config.training_data, data_transforms)
        self.test_set = ImageDataset(self.config.testing_data, data_transforms)

        self.class_name=self.train_set.classes

        return self.train_set, self.test_set
    
    def data_lode(self):
        train_loader = DataLoader(self.train_set, batch_size=8, shuffle=True)
        test_loader = DataLoader(self.test_set, batch_size=8, shuffle=False)
        return train_loader, test_loader

In [49]:
class Training:
    def __init__(self, train_loader, test_loader, train_set, test_set, config: TrainingConfig):
        self.config = config
        self.criterion = torch.nn.CrossEntropyLoss()
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.train_set = train_set
        self.test_set = test_set
        # print(self.config)
    def get_base_model(self):
        self.model = torch.load(
            self.config.updated_base_model_path
        )
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr= self.config.params_lr_rate)

    @staticmethod 
    def get_num_correct(preds, labels):
        return preds.argmax(dim=1).eq(labels).sum().item()

    def model_training(self):
        tb = SummaryWriter()
        device = ("cuda" if torch.cuda.is_available() else "cpu")
        model = self.model.to(device)
        start_time=time.time()
        for epoch in range(self.config.params_epochs):
            epoch_time=time.time()
            total_loss = 0
            total_correct = 0
            valid_correct=0
            valid_loss=0
            model.train()
            for images, labels in self.train_loader:
                images, labels = images.to(device), labels.to(device)
                preds = model(images)

                loss = self.criterion(preds, labels)
                total_loss+= loss.item()
                total_correct+= self.get_num_correct(preds, labels)

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

            tb.add_scalar("Loss", total_loss, epoch)
            tb.add_scalar("Correct", total_correct, epoch)
            tb.add_scalar("Accuracy", (total_correct/ len(self.train_set))*100, epoch)
            for name, weight in model.named_parameters():
                tb.add_histogram(name, weight, epoch)
                tb.add_histogram(f'{name}.grad',weight, epoch)
            print("Train Accuracy...")
            print("epoch:", epoch, f"total_correct: [{total_correct}/461]", "loss:",total_loss, "Accuracy", (total_correct/ len(self.train_set))*100)

            with torch.no_grad():
                model.eval()
                for images, labels in self.test_loader:
                    images, labels = images.to(device), labels.to(device)
                    preds = model(images)

                    loss = self.criterion(preds, labels)
                    valid_loss+= loss.item()
                    valid_correct+= self.get_num_correct(preds, labels)
            total_epoch_time=time.time() - epoch_time
            print("Test Accuracy...")
            print(f"total_correct: [{valid_correct}/116]", "loss:",valid_loss, "Accuracy", (valid_correct/ len(self.test_set))*100)
            print(f"The model with [{epoch}/{self.config.params_epochs}+1] epoch took {total_epoch_time/60} minutes")
        tb.close()
        total_time=time.time() - start_time
        print(f"The model with {self.config.params_epochs} epochs took {total_time/60} minutes")
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
    @staticmethod
    def save_model(path, model):
        print("saving model ...")
        torch.save(model.state_dict(), path)

In [50]:
try:
    config = ConfigurationManager()
    # prepare_callbacks_config = config.get_prepare_callback_config()
    # prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    # callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()

    data_preprocessing = Data_preprocessing(config=training_config)
    train_set, test_set = data_preprocessing.data_normalization()
    train_loader, test_loader = data_preprocessing.data_lode()

    training = Training(train_loader, test_loader, train_set, test_set, config=training_config)
    training.get_base_model()
    # training.train_valid_generator()
    training.model_training()    
except Exception as e:
    raise e

src\bird_classification\utils\config.yaml
[2023-07-24 09:39:42,351: INFO: common: created directory at: artifacts]
[2023-07-24 09:39:42,352: INFO: common: created directory at: artifacts\training]
epoch: 0 total_correct: [38/461] loss: 137.84589982032776 Accuracy 8.24295010845987
total_correct: [10/116] loss: 35.66346716880798 Accuracy 8.620689655172415
The model with [0/2] epoch took 676.8347330093384 seconds
epoch: 1 total_correct: [47/461] loss: 136.88063836097717 Accuracy 10.195227765726681
total_correct: [12/116] loss: 35.41725254058838 Accuracy 10.344827586206897
The model with [1/2] epoch took 634.8944907188416 seconds
The model with 2 epochs took 1311.7365295886993 seconds
saving model ...


In [51]:
len(train_set)

461